<a href="https://colab.research.google.com/github/htw22/htw22/blob/main/2025%EC%A4%91%EA%B0%84%EC%8B%9C%ED%97%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. NumPy 버전을 1.x (2.0 미만)으로 강제 다운그레이드합니다.
!pip install "numpy<2"

# 2. scikit-surprise 라이브러리를 설치합니다.
!pip install scikit-surprise

In [2]:
import os
import pandas as pd
import numpy as np

## 1. KMRD-small에 대해 다음 data를 구하라.

In [3]:
rates_df = pd.read_csv('rates.csv', encoding='utf-8')
rates_df.head()

,user,movie,rate,time
0,0,10003,7,1494128040
1,0,10004,7,1467529800
2,0,10018,9,1513344120
3,0,10021,9,1424497980
4,0,10022,7,1427627340


### (1) 사용자 당 평가한 영화 수

In [4]:
user_reviews_df = pd.DataFrame({
    'review_count': rates_df.groupby('user')['movie'].count(),
})
user_reviews_df.sort_values('review_count', ascending=False)[:5]

,review_count
user,
465,413
44,375
1051,322
170,278
110,262


In [5]:
user_reviews_over9_df = user_reviews_df[user_reviews_df['review_count']>9]
len(user_reviews_over9_df)

2141

### (2) 영화 당 평가 수

In [6]:
movie_reviewed_df = pd.DataFrame({
    'num_users_watch': rates_df.groupby('movie')['user'].count(),
})
movie_reviewed_df.sort_values('num_users_watch', ascending=False)[:5]

,num_users_watch
movie,
10200,7265
10048,7016
10102,4391
10016,4029
10936,3690


In [7]:
movie_reviewed_over9_df = movie_reviewed_df[movie_reviewed_df['num_users_watch']>9]
print(movie_reviewed_over9_df.sort_values('num_users_watch', ascending=False)[:5])
print(len(movie_reviewed_over9_df))

       num_users_watch
movie                 
10200             7265
10048             7016
10102             4391
10016             4029
10936             3690
517


In [8]:
movie_reviewed_over9_df.index

Index([10001, 10002, 10003, 10004, 10005, 10006, 10007, 10008, 10009, 10011,
       ...
       10970, 10971, 10975, 10979, 10980, 10981, 10983, 10988, 10994, 10998],
      dtype='int64', name='movie', length=517)

### (3) (1) & (2) 추출

In [9]:
user_movie_rates_df = rates_df[rates_df.user.isin(user_reviews_over9_df.index)]
user_movie_rates_df = user_movie_rates_df[user_movie_rates_df.movie.isin(movie_reviewed_over9_df.index)]
user_movie_rates_df.head()

,user,movie,rate,time
0,0,10003,7,1494128040
1,0,10004,7,1467529800
2,0,10018,9,1513344120
3,0,10021,9,1424497980
4,0,10022,7,1427627340


In [10]:
user_movie_rates_df.shape

(57227, 4)

### (4) (3)의 movie에 대해 movies_df 구축
- movies.txt, genres.csv 결합

In [11]:
genres_df = pd.read_csv('genres.csv', encoding='utf-8')
genres_df.head()

,movie,genre
0,10001,드라마
1,10001,멜로/로맨스
2,10002,SF
3,10002,코미디
4,10003,SF


In [12]:
movies_df = pd.read_csv('movies.txt', sep='\t', encoding='utf-8')
movies_df.head()

,movie,title,title_eng,year,grade
0,10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가
1,10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가
2,10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가
3,10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가
4,10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG


In [13]:
groups = genres_df.groupby('movie')
genres = [(list(set(x['movie'].values))[0], ','.join(x['genre'].values)) for index, x in groups]
combined_genres_df = pd.DataFrame(data=genres, columns=['movie', 'genres'])
combined_genres_df = combined_genres_df.set_index('movie')
combined_genres_df.head()

,genres
movie,
10001,"드라마,멜로/로맨스"
10002,"SF,코미디"
10003,"SF,코미디"
10004,"서부,SF,판타지,코미디"
10005,"판타지,모험,SF,액션"


In [14]:
movies_df = movies_df.set_index('movie')
movies_df.head()

,title,title_eng,year,grade
movie,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가
10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가
10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가
10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가
10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG


In [15]:
movies_df = pd.concat([movies_df, combined_genres_df], axis=1)
movies_df.head()

,title,title_eng,year,grade,genres
movie,,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,"드라마,멜로/로맨스"
10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,"SF,코미디"
10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,"SF,코미디"
10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,"서부,SF,판타지,코미디"
10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,"판타지,모험,SF,액션"


In [16]:
movies_df = movies_df[movies_df.index.isin(movie_reviewed_over9_df.index)]
movies_df.head()

,title,title_eng,year,grade,genres
movie,,,,,
10001,시네마 천국,"Cinema Paradiso , 1988",2013.0,전체 관람가,"드라마,멜로/로맨스"
10002,빽 투 더 퓨쳐,"Back To The Future , 1985",2015.0,12세 관람가,"SF,코미디"
10003,빽 투 더 퓨쳐 2,"Back To The Future Part 2 , 1989",2015.0,12세 관람가,"SF,코미디"
10004,빽 투 더 퓨쳐 3,"Back To The Future Part III , 1990",1990.0,전체 관람가,"서부,SF,판타지,코미디"
10005,스타워즈 에피소드 4 - 새로운 희망,"Star Wars , 1977",1997.0,PG,"판타지,모험,SF,액션"


In [17]:
len(movies_df)

517

In [18]:
genres_df.shape

(2025, 2)

## 2. contents-based 추천시스템

### (1) IDF를 구하기 위해 Genre 별 count
- 장르 one-hot 표현

In [19]:
genres_df = movies_df['genres'].str.get_dummies(sep=',')
genres_df.head()

,SF,가족,공포,느와르,드라마,멜로/로맨스,모험,뮤지컬,미스터리,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
movie,,,,,,,,,,,,,,,,,,,
10001,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10002,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10003,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
10004,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
10005,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1


In [20]:
genres = genres_df.columns
genres

Index(['SF', '가족', '공포', '느와르', '드라마', '멜로/로맨스', '모험', '뮤지컬', '미스터리', '범죄',
       '서부', '서사', '스릴러', '애니메이션', '액션', '에로', '전쟁', '코미디', '판타지'],
      dtype='object')

In [21]:
movie_genre = []
genre_count = dict()
for each_movie_genre in genres:
    genre_count[each_movie_genre]= genres_df[each_movie_genre].sum()

genre_count

{'SF': 45,
 '가족': 22,
 '공포': 43,
 '느와르': 9,
 '드라마': 282,
 '멜로/로맨스': 101,
 '모험': 77,
 '뮤지컬': 17,
 '미스터리': 17,
 '범죄': 65,
 '서부': 18,
 '서사': 1,
 '스릴러': 97,
 '애니메이션': 3,
 '액션': 153,
 '에로': 6,
 '전쟁': 38,
 '코미디': 106,
 '판타지': 43}

In [22]:
total_movie = len(movies_df)
genre_idf = dict()
for each_movie_genre in genres:
    genre_idf[each_movie_genre]= np.log10(total_movie/genre_count[each_movie_genre])

genre_idf

{'SF': 1.060278029318599,
 '가족': 1.3710678622717363,
 '공포': 1.080022087514356,
 '느와르': 1.7592480336546177,
 '드라마': 0.2632414347745814,
 '멜로/로맨스': 0.7091691693112999,
 '모험': 0.8269998179214606,
 '뮤지컬': 1.4830416217156686,
 '미스터리': 1.4830416217156686,
 '범죄': 0.900577186451087,
 '서부': 1.4582180379906364,
 '서사': 2.7134905430939424,
 '스릴러': 0.7267188088276977,
 '애니메이션': 2.23636928837428,
 '액션': 0.5287991122763437,
 '에로': 1.9353392927102988,
 '전쟁': 1.1337069464771323,
 '코미디': 0.6881846778291723,
 '판타지': 1.080022087514356}

In [23]:
from tqdm import tqdm

genre_idf_representation = pd.DataFrame(columns=genres, index=genres_df.index)
for index, each_row in tqdm(genres_df.iterrows()):
    dict_temp = dict()
    for each_genre in genres:
        if genres_df.loc[index][each_genre] > 0 :
            dict_temp[each_genre] = genre_idf[each_genre]
    #pdb.set_trace()
    row_to_add = pd.DataFrame(dict_temp, index=[index])
    genre_idf_representation.update(row_to_add)

genre_idf_representation

517it [00:02, 204.13it/s]


,SF,가족,공포,느와르,드라마,멜로/로맨스,모험,뮤지컬,미스터리,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
movie,,,,,,,,,,,,,,,,,,,
10001,NaN,NaN,NaN,NaN,0.263241,0.709169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10002,1.060278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.688185,NaN
10003,1.060278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.688185,NaN
10004,1.060278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.458218,NaN,NaN,NaN,NaN,NaN,NaN,0.688185,1.080022
10005,1.060278,NaN,NaN,NaN,NaN,NaN,0.827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.528799,NaN,NaN,NaN,1.080022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10981,NaN,NaN,NaN,NaN,0.263241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10983,NaN,NaN,NaN,NaN,0.263241,0.709169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10988,NaN,NaN,NaN,NaN,0.263241,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### (2) 장르기반 영화 cosine 유사도
- 각 영화의 genre IDF를 vector로 하여 영화 간의 cosine 유사도
- 영화 x 영화 유사도 matrix

In [24]:
genre_idf_representation = genre_idf_representation.fillna(0)
genre_idf_representation.head()

/tmp/ipython-input-2797825813.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  genre_idf_representation = genre_idf_representation.fillna(0)


,SF,가족,공포,느와르,드라마,멜로/로맨스,모험,뮤지컬,미스터리,범죄,서부,서사,스릴러,애니메이션,액션,에로,전쟁,코미디,판타지
movie,,,,,,,,,,,,,,,,,,,
10001,0.000000,0.0,0.0,0.0,0.263241,0.709169,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000
10002,1.060278,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.688185,0.000000
10003,1.060278,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.688185,0.000000
10004,1.060278,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.0,0.0,1.458218,0.0,0.0,0.0,0.000000,0.0,0.0,0.688185,1.080022
10005,1.060278,0.0,0.0,0.0,0.000000,0.000000,0.827,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.528799,0.0,0.0,0.000000,1.080022


In [25]:
def cos_sim_matrix(a, b):
    cos_sim = cosine_similarity(a, b)
    result_df = pd.DataFrame(data=cos_sim, index=[a.index])

    return result_df

In [26]:
from sklearn.metrics.pairwise import cosine_similarity

cs_df = cos_sim_matrix(genre_idf_representation, genre_idf_representation)
cs_df.head()

,0,1,2,3,4,5,6,7,8,9,...,507,508,509,510,511,512,513,514,515,516
movie,,,,,,,,,,,,,,,,,,,,,
10001,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.347996,0.347996,1.0,0.347996,0.739695,0.073321
10002,0.0,1.000000,1.000000,0.571580,0.493013,0.493013,0.493013,0.446496,0.493013,0.446496,...,0.000000,0.000000,0.661404,0.229166,0.000000,0.000000,0.0,0.000000,0.366373,0.000000
10003,0.0,1.000000,1.000000,0.571580,0.493013,0.493013,0.493013,0.446496,0.493013,0.446496,...,0.000000,0.000000,0.661404,0.229166,0.000000,0.000000,0.0,0.000000,0.366373,0.000000
10004,0.0,0.571580,0.571580,1.000000,0.574185,0.574185,0.574185,0.255208,0.574185,0.255208,...,0.000000,0.000000,0.378045,0.130987,0.000000,0.000000,0.0,0.000000,0.209411,0.000000
10005,0.0,0.493013,0.493013,0.574185,1.000000,1.000000,1.000000,0.581023,1.000000,0.581023,...,0.084382,0.172473,0.745404,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.427522


In [27]:
cs_df.shape

(517, 517)

### (3) 영화 평가자가 50번째로 많은 영화와 유사한 10편의 영화(제목, 장르)을 나열하라

In [28]:
movie_reviewed_df.sort_values('num_users_watch', ascending=False)[:50]

,num_users_watch
movie,
10200,7265
10048,7016
10102,4391
10016,4029
10936,3690
10001,3635
10002,3004
10071,2960
10114,2901


In [29]:
sm_list = cs_df.loc[10253].values
sm_list[0]

array([0.        , 0.        , 0.        , 0.        , 0.12183293,
       0.12183293, 0.12183293, 0.1103377 , 0.12183293, 0.1103377 ,
       1.        , 0.15630947, 0.        , 0.67790746, 0.32494299,
       0.        , 0.        , 0.36762015, 0.        , 0.3720666 ,
       0.3720666 , 0.3720666 , 0.3720666 , 0.3720666 , 1.        ,
       0.685397  , 0.45675644, 0.31886608, 0.26914514, 0.2238967 ,
       0.15059   , 0.15059   , 0.41561932, 0.        , 0.36067292,
       0.        , 0.21982606, 0.21625549, 0.        , 0.43173741,
       0.17568731, 0.        , 0.50967328, 0.54783305, 0.        ,
       0.12510651, 0.21034923, 0.        , 0.        , 0.67790746,
       0.17568731, 0.17568731, 0.17734656, 0.        , 0.36067292,
       0.45675644, 0.        , 0.        , 0.        , 0.21034923,
       0.        , 0.8868818 , 0.        , 0.37062972, 0.8868818 ,
       0.67939576, 0.        , 0.56241463, 0.        , 0.        ,
       0.        , 0.        , 0.37703159, 0.        , 0.     

In [30]:
sorted_index = np.argsort(-sm_list)[0][:15]
sorted_index

array([264, 415, 345, 191, 296,  24,  10, 373,  92, 417, 418, 204, 447,
       434, 334])

In [31]:
movies_df.loc[10253]

,10253
title,배트맨
title_eng,"Batman , 1989"
year,1990.0
grade,15세 관람가
genres,"액션,스릴러,범죄"


In [32]:
for i in np.arange(len(sorted_index)):
    index = sorted_index[i]
    print(movies_df.iloc[index].title, '\t', movies_df.iloc[index].genres, '\t', sm_list[0][index])

에프 엑스 	 액션,스릴러,범죄 	 0.9999999999999999
더티 해리 	 액션,스릴러,범죄 	 0.9999999999999999
노 머시 	 액션,스릴러,범죄 	 0.9999999999999999
배트맨 	 액션,스릴러,범죄 	 0.9999999999999999
실베스타 스텔론의 탈옥 	 액션,스릴러,범죄 	 0.9999999999999999
리썰 웨폰 	 액션,스릴러,범죄 	 0.9999999999999999
다이 하드 	 액션,스릴러,범죄 	 0.9999999999999999
이어 오브 드래곤 	 드라마,액션,스릴러,범죄 	 0.9792598633177667
첩혈쌍웅 	 드라마,액션,스릴러,범죄 	 0.9792598633177667
더티 해리 3 - 집행자 	 드라마,액션,스릴러,범죄 	 0.9792598633177667
더티 해리 4 - 써든 임팩트 	 드라마,액션,스릴러,범죄 	 0.9792598633177667
보디 히트 	 스릴러,범죄 	 0.9095386627907659
유혹은 밤 그림자처럼 	 스릴러,범죄 	 0.9095386627907659
사랑의 파도 	 드라마,스릴러,범죄 	 0.8868818044909227
블러드 심플 	 드라마,스릴러,범죄 	 0.8868818044909227


### (4) 가장 많이 출연한 배우가 출연한 영화 중 전체 영화 평점이 높은 순의 영화 5편을 추천

#### (4-1) 특정 배우(문제1의 (2)에서 가장 많이 출연한 배우)가 출연한 영화

In [33]:
castings_df = pd.read_csv('castings.csv', encoding='utf-8')
castings_df.head()

,movie,people,order,leading
0,10001,4374,1,1
1,10001,178,2,1
2,10001,3241,3,1
3,10001,47952,4,1
4,10001,47953,5,0


In [34]:
castings = [(list(set(x['people'].values))[0], x['movie'].values) for index, x in castings_df.groupby('people')]
cast_movies_df = pd.DataFrame(data=castings, columns=['people','movie'])
cast_movies_df = cast_movies_df.set_index('people')
cast_movies_df['num_movie'] = castings_df.groupby('people')['movie'].count()
cast_movies_df.head()

,movie,num_movie
people,,
5,[10062],1
9,"[10086, 10476, 10639]",3
15,"[10071, 10072, 10313, 10330, 10561, 10719, 107...",9
24,"[10336, 10514]",2
27,[10144],1


In [35]:
most_people = cast_movies_df['num_movie'].sort_values(ascending=False)[:10]
most_people

,num_movie
people,
1327,18
1843,13
1422,13
10649,11
556,11
1507,11
290,11
2248,10
535,10


In [36]:
peoples_df = pd.read_csv('peoples.txt', sep='\t', encoding='utf-8')
peoples_df.head()

,people,korean,original
0,5,아담 볼드윈,Adam Baldwin
1,8,애드리안 라인,Adrian Lyne
2,9,에이단 퀸,Aidan Quinn
3,13,구로사와 아키라,Akira Kurosawa
4,15,알 파치노,Al Pacino


In [37]:
peoples_df[peoples_df.people==1843]

,people,korean,original
705,1843,안성기,NaN


#### (4-2) 배우가 출연한 영화 목록에서 평점 순으로 {영화, 평점} 출력

In [38]:
# user_movie_rates_df 평가 pivot
user_movie_rating = user_movie_rates_df.pivot_table(index='user', columns='movie', values='rate')
user_movie_rating.shape

(2141, 517)

In [39]:
user_movie_rating.head()

movie,10001,10002,10003,10004,10005,10006,10007,10008,10009,10011,...,10970,10971,10975,10979,10980,10981,10983,10988,10994,10998
user,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,7.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN
2,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,10.0,9.0,9.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
8,10.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
movie_rate_mean = user_movie_rating.mean().to_numpy()
movie_rate_mean

array([ 9.16773785,  9.25420125,  9.09245439,  8.83630769,  8.91105121,
        8.78797468,  8.86655949,  8.47154472,  8.3       ,  5.2962963 ,
        9.09430894,  7.73913043,  8.53061224,  8.95798319,  9.0995372 ,
        8.8       ,  9.05555556,  8.91304348,  9.13617886,  8.91395349,
        8.88557214,  8.45953757,  7.53703704,  7.71854305,  7.91346154,
        7.99390244,  9.10746086,  8.77112676,  6.94736842,  8.82589286,
        8.74203822,  8.93047337,  7.9047619 ,  8.9266055 ,  9.13990162,
        8.91809524,  8.62765957,  9.15827338,  8.47474747,  8.22727273,
        8.        ,  8.375     ,  9.06891496,  8.91916168,  9.05853249,
        7.97727273,  9.00240674,  7.77777778,  8.48387097,  8.4952381 ,
        7.88291139,  7.06363636,  9.01985226,  7.10526316,  8.82300469,
        7.61450382,  7.9375    ,  8.88429752,  8.44715569,  8.50320513,
        8.4516129 ,  8.29365079,  8.10185185,  8.54545455,  9.16991888,
        9.17966538,  8.13636364,  7.85185185,  6.225     ,  7.53

In [41]:
# 영화 평점이 높은 순 영화 index
high_rate_mean_movie = np.argsort(-movie_rate_mean)
high_rate_mean_movie

array([243, 487, 284, 206, 212, 142, 462, 147, 157, 283,  94, 286, 217,
       431, 188,   1, 465,  65,  64,   0, 220,  37, 460, 268,  34,  18,
       238, 120,  82, 499,  26, 184,  14,  10,   2, 310, 507, 513,  42,
        44,  16, 299, 396,  52, 149,  81, 170, 209,  46, 148, 437, 329,
        93,  89, 357, 395,  13, 368,  31,  88, 497,  33, 402,  43,  35,
        19,  17,   4, 439, 387, 327, 141, 379,  20,  57, 293, 346,  85,
         6, 427, 309,  92,   3, 225, 307, 180, 318,  29,  54, 446, 109,
        15, 511,   5, 210, 430, 449,  27, 167, 488, 295, 364,  30, 232,
       216,  70, 101, 308, 174, 504, 383, 412, 339, 228, 248, 145, 456,
       374,  86,  36, 236,  90,  83, 205, 191, 194, 146, 433, 182, 401,
       183,  95, 415, 429, 112, 458, 335, 311, 272, 189,  79,  63, 103,
       317, 479,  12, 227, 133, 221, 302,  59, 445, 514, 155, 516,  49,
        48, 294,  38,  74,   7, 106,  21, 237, 336, 410, 485, 185,  60,
       279,  58, 199, 304,  84, 344, 474, 354, 428, 466, 378, 32

In [42]:
# 배우 1327(로버트 드 니로)가 출연한 영화
most_people_movies = cast_movies_df.loc[1843].movie
most_people_movies

array([10133, 10134, 10151, 10202, 10397, 10422, 10425, 10764, 10835,
       10837, 10841, 10843, 10900])

In [43]:
# movie_id to index
movie_id2index = dict(zip(user_movie_rating.columns, range(len(user_movie_rating.columns))))
print(user_movie_rating.columns[-1]) # 마지막 column의 movie_id)

10998


In [44]:
# 배우 1327 출연한 영화 중 rates.csv에 평점이 있는 영화 대상
import pdb
most_people_movies_rate = dict()
for i in range(len(most_people_movies)):
    i_movie = most_people_movies[i]
    if i_movie in user_movie_rating.columns:
        #pdb.set_trace()
        index = movie_id2index[i_movie]
        index_title = movies_df.iloc[index].title
        most_people_movies_rate[index_title]=movie_rate_mean[index]
        if len(most_people_movies_rate) == 5:
            break

In [45]:
most_people_movies_rate

{'고래 사냥': 8.460526315789474,
 '고래 사냥 2': 7.396551724137931,
 '바람 불어 좋은 날': 8.121212121212121,
 '이장호의 외인구단': 6.622222222222222,
 '남부군': 7.768656716417911}

In [46]:
# 평점 순으로 정렬
sored_mean_rate = sorted(most_people_movies_rate.items(), key=lambda x:x[1], reverse=True)
sored_mean_rate

[('고래 사냥', 8.460526315789474),
 ('바람 불어 좋은 날', 8.121212121212121),
 ('남부군', 7.768656716417911),
 ('고래 사냥 2', 7.396551724137931),
 ('이장호의 외인구단', 6.622222222222222)]

## 3. item 기반 추천시스템

### (1) 영화간 cosine 유사도

In [47]:
user_movie_rating = user_movie_rates_df.pivot_table(index='user', columns='movie', values='rate')

from sklearn.metrics.pairwise import cosine_similarity
user_movie_rating_t = np.transpose(user_movie_rating)
matrix_dummy = user_movie_rating_t.copy().fillna(0)
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=user_movie_rating_t.index, columns=user_movie_rating_t.index)
item_similarity.head()

movie,10001,10002,10003,10004,10005,10006,10007,10008,10009,10011,...,10970,10971,10975,10979,10980,10981,10983,10988,10994,10998
movie,,,,,,,,,,,,,,,,,,,,,
10001,1.000000,0.465176,0.309844,0.273713,0.290437,0.266802,0.288480,0.137155,0.131013,0.088799,...,0.368589,0.051052,0.038297,0.095986,0.233328,0.030856,0.041704,0.019115,0.072152,0.140646
10002,0.465176,1.000000,0.626459,0.575525,0.334692,0.322222,0.321059,0.206109,0.157439,0.111455,...,0.312571,0.105645,0.075867,0.093303,0.258214,0.041909,0.054756,0.000000,0.095238,0.131018
10003,0.309844,0.626459,1.000000,0.755276,0.304037,0.307260,0.315078,0.153697,0.131451,0.084481,...,0.256339,0.072889,0.086632,0.101314,0.245016,0.055664,0.023694,0.000000,0.064150,0.148141
10004,0.273713,0.575525,0.755276,1.000000,0.291511,0.290159,0.307700,0.145374,0.148934,0.099469,...,0.220568,0.079511,0.087637,0.078015,0.241090,0.061076,0.097379,0.000000,0.065884,0.146209
10005,0.290437,0.334692,0.304037,0.291511,1.000000,0.739626,0.705524,0.245454,0.263145,0.188463,...,0.247796,0.096412,0.115603,0.079234,0.217044,0.034335,0.042091,0.046785,0.069760,0.119568


### (2) 특정 사용자가 아직 평가하지 않은 영화 중 예상 평점이 높은 10편의 영화를 추천

In [48]:
import pdb

def CF_IBCF(user_id, movie_id):
    if movie_id in item_similarity:      # 현재 영화가 train set에 있는지 확인
        # 현재 영화와 다른 영화의 similarity 값 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 사용자의 모든 rating 값 가져오기
        user_rating = user_movie_rating_t[user_id]
        # 사용자가 평가하지 않은 영화 index 가져오기
        non_rating_idx = user_rating[user_rating.isnull()].index
        # 사용자가 평가하지 않은 영화 제거
        user_rating = user_rating.dropna()
        # 사용자가 평가하지 않은 영화의 similarity 값 제거
        sim_scores = sim_scores.drop(non_rating_idx)
        #pdb.set_trace()
        if sim_scores.sum() == 0:
            mean_rating = 3.0
        else:
            # 현 영화에 대한 예상 rating 계산, 가중치는 현 영화와 사용자가 평가한 영화의 유사도
            mean_rating = np.dot(sim_scores, user_rating) / sim_scores.sum()
    else:
        mean_rating = 3.0
    return mean_rating


In [49]:
def recommender(user, n_items=10):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = user_movie_rating.loc[user][user_movie_rating.loc[user] > 0].index    # 이미 평가한 영화 확인
    items = user_movie_rating.loc[user].drop(rated_index)
    # user가 아직 평가하지 않은 영화 대상
    for item in items.index:
        predictions.append(CF_IBCF(user, item))                   # 예상평점 계산
    #pdb.set_trace()
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]    # 예상평점이 가장 높은 영화 선택
    recommended_items = movies_df.loc[recommendations.index]['title']
    return recommended_items

In [50]:
recommender(user=110, n_items=10)

,title
movie,
10426,수업료
10318,7일간의 사랑
10354,아리랑
10526,블러드 심플
10152,자전거 도둑
10363,신사는 금발을 좋아한다
10263,돌아오지 않는 강
10432,아빠는 출장 중
10729,집시의 시간


## 4. 사용자의 평가경향을 고려한 사용자기반 협업필터링 추천시스템

### (1) user_movie_rates_df을 sklearn의 train_test_split 분리

In [51]:
## train_test_split 사용하여 rating_df를 train용과 test용으로 80:20으로 분리
##     train데이터 user_movie_rating pivot 구축, unique user_id, unique movie_id를 개수
from sklearn.model_selection import train_test_split
x = user_movie_rates_df.copy()
y = user_movie_rates_df['user']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state= 1234, stratify=y)

In [52]:
x_train.shape

(45781, 4)

In [53]:
train_user_movie_rates = x_train.pivot_table(index='user', columns='movie', values='rate').fillna(0)
train_user_movie_rates.head()

movie,10001,10002,10003,10004,10005,10006,10007,10008,10009,10011,...,10965,10970,10971,10975,10979,10980,10981,10983,10994,10998
user,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,7.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0
2,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,10.0,9.0,9.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,10.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### (2) user_similarity (peason 상관계수가 0.0 이상)

In [54]:
user_similarity_org = train_user_movie_rates.T.corr(method='pearson')
user_similarity_org.head()

user,0,2,3,4,8,9,10,13,14,18,...,28955,30368,30389,30583,31832,31919,32506,32512,32515,39759
user,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.048475,-0.003007,0.069405,0.129740,0.129232,0.142409,0.081179,0.189151,0.181191,...,-0.047508,0.009481,0.024620,0.010562,0.041015,0.016269,-0.050166,-0.060753,-0.064235,-0.053669
2,0.048475,1.000000,0.077608,0.030776,0.102507,0.103598,0.101848,-0.023484,0.064707,0.138199,...,-0.018820,0.045177,-0.016554,0.090682,-0.018711,-0.028452,-0.019873,-0.024067,-0.025446,-0.021260
3,-0.003007,0.077608,1.000000,0.018615,-0.013967,0.012059,0.134484,-0.018105,0.029515,0.055283,...,-0.049605,0.118547,0.016985,-0.052412,-0.049319,0.021967,0.141406,0.278639,0.243568,0.060258
4,0.069405,0.030776,0.018615,1.000000,0.005166,0.119870,0.010658,0.054115,0.044979,0.012433,...,0.001092,0.215557,-0.029013,0.011252,-0.040946,0.007918,-0.043487,-0.052665,-0.055683,0.004418
8,0.129740,0.102507,-0.013967,0.005166,1.000000,0.137241,0.184367,-0.011507,0.282921,0.025313,...,0.106429,-0.017472,-0.015377,-0.018471,-0.017381,-0.026430,-0.018460,-0.022356,-0.023637,-0.019749


In [55]:
user_similarity = user_similarity_org[user_similarity_org>= 0.0].fillna(0)
user_similarity.head()

user,0,2,3,4,8,9,10,13,14,18,...,28955,30368,30389,30583,31832,31919,32506,32512,32515,39759
user,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.048475,0.000000,0.069405,0.129740,0.129232,0.142409,0.081179,0.189151,0.181191,...,0.000000,0.009481,0.024620,0.010562,0.041015,0.016269,0.000000,0.000000,0.000000,0.000000
2,0.048475,1.000000,0.077608,0.030776,0.102507,0.103598,0.101848,0.000000,0.064707,0.138199,...,0.000000,0.045177,0.000000,0.090682,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.077608,1.000000,0.018615,0.000000,0.012059,0.134484,0.000000,0.029515,0.055283,...,0.000000,0.118547,0.016985,0.000000,0.000000,0.021967,0.141406,0.278639,0.243568,0.060258
4,0.069405,0.030776,0.018615,1.000000,0.005166,0.119870,0.010658,0.054115,0.044979,0.012433,...,0.001092,0.215557,0.000000,0.011252,0.000000,0.007918,0.000000,0.000000,0.000000,0.004418
8,0.129740,0.102507,0.000000,0.005166,1.000000,0.137241,0.184367,0.000000,0.282921,0.025313,...,0.106429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### (3) RMSE

In [56]:
# train 데이터의 user의 rating 평균과 영화의 평점편차 계산
rating_mean = train_user_movie_rates.mean(axis=1)   # 사용자별 영화평점 평균, axis=1이면 column에 행의 평균
rating_bias = (train_user_movie_rates.T - rating_mean).T   # 각 영화 사용자 평점 - 사용자별 영화평점평균
rating_bias

movie,10001,10002,10003,10004,10005,10006,10007,10008,10009,10011,...,10965,10970,10971,10975,10979,10980,10981,10983,10994,10998
user,,,,,,,,,,,,,,,,,,,,,
0,-0.949612,-0.949612,6.050388,6.050388,-0.949612,-0.949612,-0.949612,-0.949612,-0.949612,-0.949612,...,-0.949612,-0.949612,-0.949612,-0.949612,-0.949612,6.050388,-0.949612,-0.949612,-0.949612,-0.949612
2,9.813953,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,...,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047,-0.186047
3,-1.060078,-1.060078,-1.060078,-1.060078,8.939922,7.939922,7.939922,-1.060078,-1.060078,-1.060078,...,-1.060078,-1.060078,-1.060078,-1.060078,-1.060078,-1.060078,-1.060078,-1.060078,-1.060078,-1.060078
4,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,...,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620,-0.730620
8,9.823643,9.823643,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,...,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357,-0.176357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31919,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,...,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457,-0.329457
32506,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,...,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031,-0.124031
32512,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,...,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124,-0.246124


In [57]:
import pdb

def CF_knn_bias(user_id, movie_id, neighbor_size=10):
    if user_id not in rating_bias.index:
        prediction = 0
        return prediction
    if movie_id in rating_bias:
        # 현 user와 다른 사용자 간의 유사도 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현 movie의 평점편차 가져오기
        movie_ratings = rating_bias[movie_id].copy()
        # 현 movie에 대한 rating이 없는 사용자 삭제
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        movie_ratings = movie_ratings.drop(none_rating_idx)
        sim_scores = sim_scores.drop(none_rating_idx)

        # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
        if len(sim_scores) > 1:
            # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
            neighbor_size = min(neighbor_size, len(sim_scores))
            # array로 바꾸기 (argsort를 사용하기 위함)
            sim_scores = np.array(sim_scores)
            movie_ratings = np.array(movie_ratings)
            # 유사도를 순서대로 정렬
            user_idx = np.argsort(sim_scores)
            # 유사도와 rating을 neighbor size만큼 받기
            sim_scores = sim_scores[user_idx][-neighbor_size:]
            movie_ratings = movie_ratings[user_idx][-neighbor_size:]
            if sim_scores.sum() == 0:    # user_id와 유사도가 0인 경우 있음.
                prediction = rating_mean[user_id]
            else:
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
                # 예측값에 현 사용자의 평균 더하기
                prediction = prediction + rating_mean[user_id]
        else:
            prediction = rating_mean[user_id]
    else:
        prediction = rating_mean[user_id]
    return prediction

In [58]:
id_pairs = zip(x_test['user'], x_test['movie'])
y_pred = np.array([CF_knn_bias(user, movie, neighbor_size=10) for (user, movie) in id_pairs])
y_true = np.array(x_test['rate'])

In [59]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [60]:
print(RMSE(y_true, y_pred))

7.267358213826648


### (4) 영화를 200개 이상 평가한 사용자 list

In [61]:
user_200_review = list(user_reviews_df[user_reviews_df["review_count"] >= 200].index)
len(user_200_review)

12

In [62]:
user_200_review

[44, 92, 95, 110, 146, 170, 224, 465, 1051, 1662, 1820, 2769]

### (4) test data에서 user_200_review의 true_user2items

In [63]:
test_user_movie_rates = x_test.pivot_table(index='user', columns='movie', values='rate').fillna(0)
test_user_movie_rates.head()

movie,10001,10002,10003,10004,10005,10006,10007,10008,10009,10011,...,10970,10971,10975,10979,10980,10981,10983,10988,10994,10998
user,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
test_user_movie_rates.shape

(2141, 490)

In [65]:
test_user_movie_rates.loc[2]

,2
movie,
10001,0.0
10002,0.0
10003,0.0
10004,0.0
10005,0.0
...,...
10981,0.0
10983,0.0
10988,0.0


In [66]:
true_user2items = dict()
top_n = 30

for user in user_200_review:
    # 각 사용자의 평점 시리즈를 가져온 후, 점수 높은 순으로 정렬
    test_user_ratings = test_user_movie_rates.loc[user].sort_values(ascending=False)[:top_n]

    # pdb.set_trace()
    # 점수가 높은 순서로 상위 10개의 영화 선택
    top_10_movies = list(test_user_ratings.index)

    # 사용자 ID를 키로 하고, 상위 10개의 영화 리스트를 값으로 저장
    true_user2items[user] = top_10_movies

true_user2items

{44: [10001,
  10250,
  10035,
  10614,
  10238,
  10242,
  10107,
  10149,
  10148,
  10249,
  10253,
  10213,
  10588,
  10741,
  10744,
  10073,
  10273,
  10115,
  10114,
  10110,
  10936,
  10046,
  10970,
  10016,
  10185,
  10004,
  10012,
  10020,
  10048,
  10072],
 92: [10301,
  10701,
  10405,
  10262,
  10272,
  10909,
  10871,
  10329,
  10718,
  10173,
  10045,
  10721,
  10421,
  10299,
  10964,
  10271,
  10921,
  10025,
  10142,
  10563,
  10161,
  10761,
  10008,
  10815,
  10215,
  10298,
  10171,
  10007,
  10710,
  10422],
 95: [10001,
  10111,
  10034,
  10037,
  10058,
  10065,
  10102,
  10103,
  10110,
  10113,
  10021,
  10116,
  10200,
  10262,
  10440,
  10609,
  10700,
  10746,
  10028,
  10998,
  10020,
  10004,
  10016,
  10018,
  10238,
  10149,
  10519,
  10118,
  10314,
  10027],
 110: [10035,
  10014,
  10741,
  10047,
  10095,
  10217,
  10314,
  10670,
  10382,
  10021,
  10019,
  10251,
  10007,
  10117,
  10012,
  10758,
  10200,
  10409,
  10847,

### (5) neighbor 30의 평가를 기준으로 평가 점수 상위 30개 pred_user2items

In [67]:
# 각 사용자에 대해 neighbor 30명을 기준으로 예측 평점 구하고, 상위 10개의 영화 추천
pred_user2items = {}
neighbor_size = 30
top_n = 30  # 추천할 영화 개수

for user in user_200_review:  # user_ids는 평가할 사용자 목록
    # 각 사용자별 평가하지 않은 movie_id
    non_rating_movi_ids = list(train_user_movie_rates.loc[user][train_user_movie_rates.loc[user]==0].index)
    user_predictions = {}
    for movie in non_rating_movi_ids:
        # CF_knn_bias 함수 호출하여 예측 평점 계산
        predicted_rating = CF_knn_bias(user, movie, neighbor_size=neighbor_size)
        user_predictions[movie] = predicted_rating

    # 예측 평점을 기준으로 상위 10개의 영화 추천
    top_movies = sorted(user_predictions, key=user_predictions.get, reverse=True)[:top_n]
    pred_user2items[user] = top_movies

pred_user2items

{44: [10039,
  10029,
  10037,
  10007,
  10005,
  10061,
  10800,
  10751,
  10062,
  10128,
  10105,
  10066,
  10217,
  10049,
  10055,
  10019,
  10519,
  10025,
  10420,
  10024,
  10290,
  10453,
  10095,
  10042,
  10410,
  10106,
  10178,
  10008,
  10026,
  10056],
 92: [10016,
  10071,
  10048,
  10200,
  10238,
  10001,
  10038,
  10102,
  10330,
  10800,
  10003,
  10035,
  10046,
  10970,
  10489,
  10793,
  10110,
  10047,
  10566,
  10962,
  10004,
  10173,
  10012,
  10114,
  10037,
  10002,
  10688,
  10249,
  10253,
  10440],
 95: [10021,
  10058,
  10238,
  10018,
  10504,
  10173,
  10061,
  10004,
  10006,
  10007,
  10065,
  10742,
  10047,
  10023,
  10217,
  10262,
  10113,
  10251,
  10185,
  10062,
  10546,
  10005,
  10519,
  10697,
  10751,
  10828,
  10149,
  10055,
  10025,
  10056],
 110: [10200,
  10071,
  10021,
  10012,
  10034,
  10253,
  10033,
  10314,
  10035,
  10640,
  10561,
  10047,
  10007,
  10019,
  10453,
  10117,
  10095,
  10106,
  10519,

### (6) precision@10, recal@10

In [68]:
import pdb
def precision_recall_at_k(true_user2items, pred_user2items, k=10):
    precisions = []
    recalls = []

    for user, true_items in true_user2items.items():
        pred_items = pred_user2items.get(user, [])[:k]  # 상위 k개 예측 항목

        # 교집합의 크기 (정확히 맞춘 예측)
        hit_count = len(set(true_items) & set(pred_items))

        # Precision@k: 예측한 상위 k개 중 실제로 본 항목의 비율
        precision = hit_count / min(len(pred_items), k) if pred_items else 0
        precisions.append(precision)

        # Recall@k: 실제로 본 항목 중 예측한 상위 k개에 포함된 항목의 비율
        recall = hit_count / len(true_items) if any(true_items) else 0
        recalls.append(recall)
        #pdb.set_trace()

    # 전체 사용자에 대한 평균 precision@k와 recall@k
    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0

    return avg_precision, avg_recall

precision_at_10, recall_at_10 = precision_recall_at_k(true_user2items, pred_user2items, k=10)
print("Precision@10:", precision_at_10)
print("Recall@10:", recall_at_10)

Precision@10: 0.425
Recall@10: 0.14166666666666666


# Surprise 비교

In [69]:
# 1. Surprise 라이브러리 및 필요 모듈 임포트
from surprise import Reader, Dataset, KNNWithMeans
from surprise import accuracy
import time # 실행 시간 측정을 위해

# 2. 중간고사 데이터를 Surprise 형식으로 변환
# 'rates.csv'의 평점(rate)은 1-10점 스케일
reader = Reader(rating_scale=(1, 10))

# x_train DataFrame을 Surprise Dataset으로 로드
data_train = Dataset.load_from_df(x_train[['user', 'movie', 'rate']], reader)

# fit() 메서드를 위한 full trainset 빌드
trainset = data_train.build_full_trainset()

# x_test를 (user, item, true_rating) 튜플 리스트로 변환
testset = list(x_test[['user', 'movie', 'rate']].itertuples(index=False, name=None))


# 3. Surprise KNNWithMeans 모델 학습 및 RMSE 계산
# [TO-DO 1] Surprise RMSE 계산

# 10주-2 실습 옵션 (pearson_baseline, user_based)
sim_options = {'name': 'pearson_baseline', 'user_based': True}

# KNNWithMeans 알고리즘 정의 (k=30, 중간고사와 동일하게 설정, 불필요한 출력 제거)
algo = KNNWithMeans(k=30, sim_options=sim_options, verbose=False)

# 모델 학습
start_fit = time.time()
algo.fit(trainset)
end_fit = time.time()
print(f"Surprise 모델 학습 소요 시간: {end_fit - start_fit:.2f}초")

# Testset으로 예측 수행
predictions = algo.test(testset)

# RMSE 계산 및 출력 (보고서 4-(1) 표에 기입)
rmse_surprise = accuracy.rmse(predictions, verbose=False)
print(f"[TO-DO 1] Surprise KNNWithMeans RMSE 결과: {rmse_surprise:.4f}")


# 4. Surprise 모델로 P@10, R@10용 예측 리스트 생성
# [TO-DO 2, 3] P@10, R@10 계산

# 예측 결과를 저장할 딕셔너리
pred_user2items_surprise = {}
top_n = 30 # 상위 30개 (중간고사 4-(5)와 동일)

start_pred = time.time()

# 중간고사 4-(5) [178]번 셀의 로직과 동일하게 반복
for user in user_200_review:
    # 사용자가 평가하지 않은 영화 ID 목록 (trainset 기준, 0으로 채워진)
    non_rating_movi_ids = list(train_user_movie_rates.loc[user][train_user_movie_rates.loc[user] == 0].index)
    user_predictions = {}

    for movie in non_rating_movi_ids:
        # [핵심] 수동 구현 함수(CF_knn_bias) 대신, 학습된 algo 모델의 predict() 메서드 사용
        predicted_rating = algo.predict(uid=user, iid=movie).est
        user_predictions[movie] = predicted_rating

    # 예측 평점을 기준으로 상위 top_n개의 영화 추천
    top_movies = sorted(user_predictions, key=user_predictions.get, reverse=True)[:top_n]
    pred_user2items_surprise[user] = top_movies

end_pred = time.time()
print(f"P@10/R@10 예측 리스트 생성 소요 시간: {end_pred - start_pred:.2f}초")


# 5. P@10, R@10 최종 계산 및 출력
# precision_recall_at_k 함수는 중간고사 4-(6) [180]번 셀에서 이미 정의된 것을 사용
precision_at_10, recall_at_10 = precision_recall_at_k(true_user2items, pred_user2items_surprise, k=10)

# P@10, R@10 결과 출력 (보고서 4-(2) 표에 기입)
print(f"[TO-DO 2] Surprise 모델 Precision@10: {precision_at_10:.4f}")
print(f"[TO-DO 3] Surprise 모델 Recall@10: {recall_at_10:.4f}")

Surprise 모델 학습 소요 시간: 1.51초
[TO-DO 1] Surprise KNNWithMeans RMSE 결과: 1.8352
P@10/R@10 예측 리스트 생성 소요 시간: 0.26초
[TO-DO 2] Surprise 모델 Precision@10: 0.0583
[TO-DO 3] Surprise 모델 Recall@10: 0.0194
